In [3]:
from pathlib import Path
from urllib.parse import urlparse
import requests
import json
from PIL import Image
from io import BytesIO
from matplotlib import patches
import matplotlib.pyplot as plt


subscription_key = 'f9bee9c0e51545bab4e932a5cdf27ff4'
face_api_url = 'https://mycogser1.cognitiveservices.azure.com/face/v1.0/detect'
face_api_url_verify = 'https://mycogser1.cognitiveservices.azure.com/face/v1.0/verify'

def annotate_image(image_url, subscription_key, api_url, show_face_id=False):

    # The default header must include the sunbscription key
    headers = {'Ocp-Apim-Subscription-Key': subscription_key}

    params = {
        'returnFaceId': 'true',
        'returnFaceLandmarks': 'false',
        'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
    }

    # Figure out if this is a local file or url
    parsed_url = urlparse(image_url)
    if parsed_url.scheme == 'file':
        image_data = open(parsed_url.path, "rb").read()

        # When making the request, we need to add a Content-Type Header
        # and pass data instead of a url
        headers['Content-Type']='application/octet-stream'
        response = requests.post(api_url, params=params, headers=headers, data=image_data)

        # Open up the image for plotting
        image = Image.open(parsed_url.path)
    else:
        # Pass in the URL to the API
        response = requests.post(api_url, params=params, headers=headers, json={"url": image_url})
        image_file = BytesIO(requests.get(image_url).content)
        image = Image.open(image_file)

    faces = response.json()

    fig, ax = plt.subplots(figsize=(10,10))

    ax.imshow(image, alpha=0.6)
    for face in faces:
        fr = face["faceRectangle"]
        fa = face["faceAttributes"]
        origin = (fr["left"], fr["top"])
        p = patches.Rectangle(origin, fr["width"],
                            fr["height"], fill=False, linewidth=2, color='b')
        ax.axes.add_patch(p)
        ax.text(origin[0], origin[1], "%s, %d"%(fa["gender"].capitalize(), fa["age"]),
                fontsize=16, weight="bold", va="bottom")

        if show_face_id:
            ax.text(origin[0], origin[1]+fr["height"], "%s"%(face["faceId"][:5]),
            fontsize=12, va="bottom")
    ax.axis("off")

    # Explicitly closing image so it does not show in the notebook
    plt.close()
    return fig, faces




In [1]:
def face_compare(id_1, id_2, api_url):
    
    headers = {
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': subscription_key
    }

    body = {"faceId1": id_1, "faceId2": id_2}

    params = {}
    response = requests.post(api_url,
                            params=params,
                            headers=headers,
                            json=body)
    return response.json()

cousins_image, cousins = annotate_image('https://i.postimg.cc/vBQTs1XH/cousins-image.jpg',
                                            subscription_key,
                                            face_api_url,
                                            show_face_id=True)

my_solo_image, me = annotate_image('https://i.postimg.cc/d0Ft27bm/my-solo-image.jpg',
                                           subscription_key,
                                           face_api_url,
                                           show_face_id=True)

cousins_image 

my_solo_image

NameError: name 'annotate_image' is not defined

In [4]:
print("FaceIDs and corresponding ages of the Faces in the cousins image")
print("faceid0= ",cousins[0]['faceId'],"AGE= ",cousins[0]['faceAttributes']['age'])
print("faceid1= ",cousins[1]['faceId'],"AGE= ",cousins[1]['faceAttributes']['age'])
print("faceid2= ",cousins[2]['faceId'],"AGE= ",cousins[2]['faceAttributes']['age'])
print("faceid3= ",cousins[3]['faceId'],"AGE= ",cousins[3]['faceAttributes']['age'])
print("faceid4= ",cousins[4]['faceId'],"AGE= ",cousins[4]['faceAttributes']['age'])
print("")
print("FaceID and corresponding age of the Face in my solo image")
print("faceid_myface= ",me[0]['faceId'],"AGE= ",me[0]['faceAttributes']['age'])



FaceIDs and corresponding ages of the Faces in the cousins image
faceid0=  e4908769-db9b-449a-aa2c-0cf7a6d83928 AGE=  25.0
faceid1=  9b47b03e-6a14-4549-8748-bcbd2a2f1edf AGE=  14.0
faceid2=  2cccdf9f-bbdd-4d6e-bd77-aa3c9c5e1c18 AGE=  29.0
faceid3=  5e874513-66fa-4bf5-b852-a7c5552cd1a3 AGE=  20.0
faceid4=  cd8a1a81-f63e-425e-a95d-a563eba055d1 AGE=  16.0

FaceID and corresponding age of the Face in my solo image
faceid_myface=  8f80465a-34eb-4656-9d11-7704f83a54af AGE=  21.0


In [6]:
for i in range(5):
    print("Comparing faceid",i," with my face: ")
    print(face_compare(me[0]['faceId'], cousins[i]['faceId'], face_api_url_verify))

Comparing faceid 0  with my face: 
{'isIdentical': False, 'confidence': 0.30385}
Comparing faceid 1  with my face: 
{'isIdentical': False, 'confidence': 0.09449}
Comparing faceid 2  with my face: 
{'isIdentical': True, 'confidence': 0.69045}
Comparing faceid 3  with my face: 
{'isIdentical': False, 'confidence': 0.16735}
Comparing faceid 4  with my face: 
{'isIdentical': False, 'confidence': 0.28128}
